# Now that we have cleaned the dataset we need to do some embedding for the query and the code snippet to share the same vector space ( And therefore apply cos similarity)

In [1]:
import numpy as np
import math
import random
import tensorflow as tf
import torch
import csv
import transformers
from transformers import  AlbertConfig,TFAlbertModel
from transformers import AlbertTokenizer
from transformers import  AlbertConfig,TFAlbertModel

In [2]:
from transformers import AlbertTokenizer, TFAlbertModel

albert_tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

In [3]:
from transformers import AlbertConfig, AlbertForPreTraining, load_tf_weights_in_albert

In [4]:
# Loading the weights and the configuration of our trained ALBERT model
config = AlbertConfig.from_pretrained('./albert', output_hidden_states=True)

model = TFAlbertModel.from_pretrained('./albert', config=config,  from_pt=True)
print(model.config)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertModel: ['predictions.LayerNorm.bias', 'predictions.bias', 'albert.embeddings.position_ids', 'sop_classifier.classifier.bias', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.dense.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing TFAlbertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFAlbertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on,

AlbertConfig {
  "_name_or_path": "./albert",
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.3.3",
  "type_vocab_size": 2,
  "vocab_size": 30000
}



In [5]:
import pandas as pd

In [6]:
train_df = pd.read_csv('train_sorted.csv')

In [7]:
train_df.head()

,Unnamed: 0,repo,path,function_name,lineno,original_function,function_tokens,docstring_tokens,url,function_tokens_count
0,89542,espressif/esptool,ecdsa/numbertheory.py,gcd2,1,"def gcd2(a, b):\n """"""Greatest common diviso...",while return,greatest common divisor using euclid s algorithm,https://github.com/espressif/esptool/blob/mast...,2
1,74655,gwastro/pycbc,pycbc/conversions.py,chi_a,1,"def chi_a(mass1, mass2, spin1z, spin2z):\n ...",chi return,returns the aligned mass weighted spin differe...,https://github.com/gwastro/pycbc/blob/master/p...,2
2,165400,SHDShim/pytheos,pytheos/eqn_bm3.py,bm3_k,1,"def bm3_k(p, v0, k0, k0p):\n """"""\n calcu...",return cal,calculate bulk modulus wrapper for cal k bm3 c...,https://github.com/SHDShim/pytheos/blob/master...,2
3,177346,olsoneric/pedemath,pedemath/vec2.py,cross_v2,1,"def cross_v2(vec1, vec2):\n """"""Return the c...",cross return,return the crossproduct of the two vectors as ...,https://github.com/olsoneric/pedemath/blob/mas...,2
4,285898,asweigart/pyautogui,pyautogui/__init__.py,getPointOnLine,1,"def getPointOnLine(x1, y1, x2, y2, n):\n """"...",getpointonline return,returns the x y tuple of the point that has pr...,https://github.com/asweigart/pyautogui/blob/ma...,2


In [8]:
print(len(train_df['docstring_tokens'].values))

303256


In [10]:
training_head = train_df.head(100)

# IMPORTANT : the embedding phase can take a long time if u wish to stop, stop it via the keyboard interrupt so you can get the list of words already embedded and not go back from scratch

In [13]:
avg_embeddings = []
for count,item in enumerate(train_df['docstring_tokens'].values): #traverse thorugh all train data set docstrings
    e = albert_tokenizer.encode(item, max_length=512)
    input = tf.constant(e)[None, :]  # Batch size 1 
    output = model(input)
    v = [0]*768
    for i in range(1, len(input[0])-1):
        v = v + output[0][0][i].numpy()  # generate sentence vectors by averaging the word vectors
    avg_embeddings.append(v/len(input[0])) #append all sentence vectors into a list
    


KeyboardInterrupt: 

# Save it in a "tsv" file, tsv files are shared vector representation files

In [19]:
with open("avg_embeddings.tsv","w+",newline='') as my_csv:
    csvWriter = csv.writer(my_csv,delimiter='\t')
    csvWriter.writerows(avg_embeddings)

In [14]:
print("test")

test


In [18]:
print(avg_embeddings[0])

[-4.16839517e-02  6.88348025e-01  6.39434850e-01 -4.71354170e-01
 -4.03322669e-01  7.85256037e-02  1.16616547e-01 -3.81152095e-01
  1.75100972e-01 -3.96167673e-01  3.81783211e-01  1.36544413e+00
  9.62476909e-01 -3.75559979e-01 -1.16728918e+00 -4.25497208e-01
 -3.73788972e-01 -2.11227715e-01 -4.07284307e-01  2.28629363e+00
 -3.38343913e-01 -2.32977846e-01  9.42843235e-01  4.41283967e-01
  4.94421538e-01 -9.63364097e-01 -2.40779852e-01  1.55508692e-01
  1.45130007e+00 -2.54643968e-01  3.33144273e-01 -4.86735974e-01
 -5.08385569e-01 -4.22468544e-01 -7.63946547e-01  1.76803382e-01
  4.38505408e-02 -5.78720345e-01  8.17443598e-01 -8.25267839e-02
  4.59012662e-01  7.93389394e-01 -1.84188743e+00 -7.38303846e-01
  4.77419954e-01  7.41540783e-01 -5.93035770e-01 -7.14947515e-01
 -6.70461059e-01 -1.55738020e+00  4.82279084e-01  1.52183481e+00
 -9.43557973e-01 -6.79263050e-01  4.77522809e-01 -5.75293267e-01
 -8.68631369e-02  1.80469356e-01  8.26092188e-01 -1.13392010e+00
  6.04530831e-01 -2.87263